# Fine-Tune Qwen3-8B for Financial Extraction (Colab)

This notebook fine-tunes Qwen3-8B-Instruct using QLoRA and Unsloth for structured financial entity extraction from SEC 10-K reports.

**Requirements**: T4 GPU (free tier) or A100 (Colab Pro)

## 1. Install Dependencies

In [ ]:
%%capture
!pip install unsloth
!pip install --no-deps trl peft accelerate bitsandbytes
!pip install datasets pydantic loguru

## 2. Upload Training Data

Upload your `train.jsonl` file or clone your repo.

In [ ]:
# Option A: Clone your GitHub repo
!git clone https://github.com/ineedmoney527/fine-tuning-sec-fillings.git
%cd fine-tuning-sec-fillings

# Option B: Upload manually (uncomment below)
# from google.colab import files
# uploaded = files.upload()  # Upload train.jsonl

## 3. Load Model with Unsloth

In [ ]:
from unsloth import FastLanguageModel
from unsloth.chat_templates import get_chat_template

# Configuration
MODEL_NAME = "unsloth/Qwen3-8B-Instruct-unsloth-bnb-4bit"
MAX_SEQ_LENGTH = 8192

# Load model with 4-bit quantization
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_NAME,
    max_seq_length=MAX_SEQ_LENGTH,
    dtype=None,  # Auto-detect
    load_in_4bit=True,
)

print(f"Model loaded: {MODEL_NAME}")

## 4. Configure LoRA Adapters

In [ ]:
# Add LoRA adapters
model = FastLanguageModel.get_peft_model(
    model,
    r=16,              # LoRA rank
    lora_alpha=32,     # Scaling factor
    lora_dropout=0.05,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj"
    ],
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=42,
)

# Set up chat template
tokenizer = get_chat_template(tokenizer, chat_template="qwen-2.5")
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print("LoRA adapters configured")

## 5. Load Training Data

In [ ]:
import json
from datasets import Dataset

# Load JSONL data
DATA_PATH = "data/train.jsonl"  # Adjust path if needed

data = []
with open(DATA_PATH, 'r') as f:
    for line in f:
        if line.strip():
            data.append(json.loads(line))

dataset = Dataset.from_list([{"messages": ex["messages"]} for ex in data])
print(f"Loaded {len(dataset)} training examples")

## 6. Configure Training

In [ ]:
from trl import SFTTrainer, SFTConfig

def formatting_func(examples):
    texts = []
    for messages in examples["messages"]:
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=False
        )
        texts.append(text)
    return texts

# Training configuration
training_args = SFTConfig(
    output_dir="./outputs",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    warmup_steps=5,
    num_train_epochs=3,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=1,
    save_strategy="epoch",
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="linear",
    seed=42,
    max_seq_length=MAX_SEQ_LENGTH,
    packing=False,
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    args=training_args,
    formatting_func=formatting_func,
)

print("Trainer configured")

## 7. Train!

In [ ]:
trainer_stats = trainer.train()
print(f"Training complete! Final loss: {trainer_stats.training_loss:.4f}")

## 8. Save Model

In [ ]:
# Save LoRA adapter
model.save_pretrained("./outputs/qwen3-8b-financial-lora")
tokenizer.save_pretrained("./outputs/qwen3-8b-financial-lora")
print("Model saved to ./outputs/qwen3-8b-financial-lora")

## 9. Download Model (Optional)

In [ ]:
# Zip and download the adapter
!zip -r qwen3-financial-lora.zip ./outputs/qwen3-8b-financial-lora

from google.colab import files
files.download('qwen3-financial-lora.zip')

## 10. Test Inference

In [ ]:
# Enable inference mode
FastLanguageModel.for_inference(model)

# Test prompt
test_messages = [
    {"role": "system", "content": "Extract financial metrics as JSON."},
    {"role": "user", "content": "Revenue: $100 million. Net Income: $10 million. Total Assets: $500 million."}
]

inputs = tokenizer.apply_chat_template(
    test_messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

outputs = model.generate(
    input_ids=inputs,
    max_new_tokens=256,
    temperature=0.1,
)

response = tokenizer.decode(outputs[0][inputs.shape[1]:], skip_special_tokens=True)
print("Model output:")
print(response)